In [1]:
import pyarrow.parquet as pq
import pandas as pd
import os

In [2]:
# column rename dictionaries
yellow_col_dict = {"tpep_pickup_datetime": "Pickup_datetime",
                   "tpep_dropoff_datetime": "Dropoff_datetime",
                   }

green_col_dict = {"lpep_pickup_datetime": "Pickup_datetime",
                   "lpep_dropoff_datetime": "Dropoff_datetime",
                   }

fhv_col_dict = {"pickup_datetime": "Pickup_datetime",
                "dropOff_datetime": "Dropoff_datetime",
                "PUlocationID": "PULocationID",
                "DOlocationID": "DOLocationID",
                }

# key_columns
yellow_or_green_columns = ["Vehicle_type",
                        "Pickup_datetime",
                        "Dropoff_datetime",
                        "passenger_count",
                        "trip_distance",
                        "PULocationID",
                        "DOLocationID",
                        "RatecodeID",
                        "payment_type",
                        "fare_amount",
                        "extra",
                        "total_amount",
                        "congestion_surcharge"]

fhv_columns = ["Vehicle_type",
            "Pickup_datetime",
            "Dropoff_datetime",
            "PULocationID",
            "DOLocationID",
            "SR_Flag"]

In [3]:
def get_dataframe_from_parquet_file(file_name: str) -> pd.DataFrame:
    """Takes a parquet file name, returns a dataframe.
    
    Takes: str
    
    Returns: pd.DataFrame
    """

    data = pq.read_table(f'../data/{file_name}')
    df: pd.DataFrame = data.to_pandas()

    print(df.columns)
    
    if "yellow" in file_name:
        # case 1: Yellow Taxi
        df["Vehicle_type"] = "Yellow"
        df.rename(columns=yellow_col_dict, inplace=True)
        df = df[yellow_or_green_columns]

    elif "green" in file_name:
        # case 2: Green Taxi
        df["Vehicle_type"] = "Green"
        df.rename(columns=green_col_dict, inplace=True)
        df = df[yellow_or_green_columns]

    else:
        # case 3: For Hire Vehicle
        df["Vehicle_type"] = "FHV"
        df.rename(columns=fhv_col_dict, inplace=True)
        df = df[fhv_columns]

    return df

def get_all_trip_data_in_data_directory(year_month: str) -> pd.DataFrame:
    """Returns trip data of a particular year/month in the as a single dataframe.
    
    Takes: str
    
    Returns: pd.DataFrame
    """
    
    final_df = pd.DataFrame()

    for file in os.listdir("../data"):
        if year_month in file:
            print(file)
            df = get_dataframe_from_parquet_file(file)

            final_df = pd.concat([final_df, df])

    return final_df

dec_df = get_all_trip_data_in_data_directory(year_month="2023-12")

# print(dec_df.head())
# dec_df.to_csv("dec.csv")

yellow_tripdata_2023-12.parquet
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')
fhv_tripdata_2023-12.parquet
Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')
green_tripdata_2023-12.parquet
Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'conges

In [4]:
print(dec_df.head())
dec_df.head().to_csv("head.csv")

  Vehicle_type     Pickup_datetime    Dropoff_datetime  passenger_count  \
0       Yellow 2023-12-01 00:06:06 2023-12-01 00:15:47              0.0   
1       Yellow 2023-12-01 00:22:26 2023-12-01 00:28:53              0.0   
2       Yellow 2023-12-01 00:59:44 2023-12-01 01:13:22              2.0   
3       Yellow 2023-12-01 00:22:17 2023-12-01 00:30:59              1.0   
4       Yellow 2023-12-01 00:18:16 2023-12-01 00:25:32              2.0   

   trip_distance  PULocationID  DOLocationID  RatecodeID  payment_type  \
0           1.10         230.0          48.0         1.0           1.0   
1           1.50         142.0         238.0         1.0           1.0   
2           2.20         114.0         186.0         1.0           1.0   
3           0.66          79.0          79.0         1.0           2.0   
4           2.20         229.0         263.0         1.0           1.0   

   fare_amount  extra  total_amount  congestion_surcharge  SR_Flag  
0         10.0    3.5         16.50

In [12]:
# Part 1: Feature Engineering

# calculate trip duration in minutes based on pickup and dropoff timestamps
dec_df["Pickup_datetime"] = pd.to_datetime(dec_df["Pickup_datetime"])
dec_df["Dropoff_datetime"] = pd.to_datetime(dec_df["Dropoff_datetime"])

dec_df["trip_duration_in_mins"] = round((dec_df["Dropoff_datetime"] - dec_df["Pickup_datetime"]) / pd.Timedelta(minutes=1), 2)

# check result
dec_df[["Pickup_datetime", "Dropoff_datetime", "trip_duration_in_mins"]].head().to_csv("duration.csv")


# Data Cleaning

# filter out erroneous entries such as negative total fare amount and zero duration trips
filtered_df = dec_df[(dec_df["total_amount"] >= 0) & (dec_df["trip_duration_in_mins"] > 0)]

# TODO get Dec 2023 NYC Westher data
# TODO Join with External Datasets:
# Enrich the taxi trip data by joining it with external datasets, such as weather data,
# demographic data, event calendars, and public transportation schedules.
# This can provide additional context and insights into factors influencing taxi usage.




4817530
40507


In [ ]:
# Part 2: Data Transformation

# Part 1: Data Cleaning

# TODO Data Cleaning and Filtering
# Clean and filter the data to remove outliers, erroneous entries, or incomplete records. This may involve removing trips with unrealistic values (e.g., negative fares, zero duration),
# correcting data inconsistencies, and handling missing or invalid values.

# TODO Date and Time Parsing:
# Extract meaningful date and time components from the timestamp data, such as year, month, day of the week, hour of the day, and time of day (e.g., morning, afternoon, evening).
# This can help analyze temporal patterns in taxi usage.


# Part 3: Data Processing & Aggregate Statistics

# TODO Aggregate Statistics:
# Calculate summary statistics for various aspects of taxi trips, such as total number of trips,
# average fare amount, average trip duration, and total distance traveled.
# Aggregate the data at different levels of granularity (e.g., hourly, daily, weekly) to analyze trends over time.

# TODO Customer Segmentation:
# Segment taxi trips based on customer characteristics such as trip purpose
# (e.g., airport transfers, commuting, leisure), passenger count, and payment method.
# This can help understand customer preferences and behavior.

# TODO Time Series Analysis:
# Apply time series analysis techniques to analyze temporal trends in taxi usage,
# such as daily and weekly patterns, seasonality, and long-term trends.
# This can help forecast future demand and optimize taxi fleet management.